<a name="about"></a>
About this notebook
======

This notebook assumes you have ran the local Census Regression notebook and you have not deleted the LOCAL_ROOT folder. In this notebook, we will use online prediction on a pre-trained Tensorflow model. This notebook will does not assume that the notebook "3. Census Regression Cloud Training" was executed.

<a name="setup"></a>
Setting things up
=====

In [1]:
import datalab_structured_data as sd

Lets look at the versions of structured_data and TF we have. Make sure TF is 1.0.0, and SD is 0.0.1.

In [2]:
import os
import pandas as pd

import tensorflow as tf
from tensorflow.python.lib.io import file_io

print('tf ' + str(tf.__version__))
print('sd ' + str(sd.__version__))

tf 1.0.0
sd 0.0.1


This notebook will write files during prediction. Please give a root folder you wish to use.

In [3]:
LOCAL_ROOT = './census_regression_workspace' # This should be the same as what was used in the local census notebook
CLOUD_ROOT = 'gs://' + datalab_project_id() + '-census-regression-datalab'

if not file_io.file_exists(LOCAL_ROOT):
  raise ValueError('LOCAL_ROOT not found. Did you run the local notebook?')
!gsutil mb {CLOUD_ROOT}

Creating gs://cloud-ml-dev-census-regression-datalab/...
ServiceException: 409 Bucket cloud-ml-dev-census-regression-datalab already exists.


First, let us put the csv files on GCS and the output of training.

In [4]:
!gsutil -m cp {os.path.join(LOCAL_ROOT, '*_data.csv')} {CLOUD_ROOT}
!gsutil -m cp -r {os.path.join(LOCAL_ROOT, 'training')} {CLOUD_ROOT}

Copying file://./census_regression_workspace/eval_data.csv [Content-Type=text/csv]...
Copying file://./census_regression_workspace/predict_data.csv [Content-Type=text/csv]...
Copying file://./census_regression_workspace/train_data.csv [Content-Type=text/csv]...
/ [3/3 files][200.1 KiB/200.1 KiB] 100% Done                                    
Operation completed over 3 objects/200.1 KiB.                                    
Copying file://./census_regression_workspace/training/model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/assets.extra/transforms.json [Content-Type=application/json]...
Copying file://./census_regression_workspace/training/model/a

In [5]:
!gsutil ls {CLOUD_ROOT}/training

gs://cloud-ml-dev-census-regression-datalab/training/evaluation_model/
gs://cloud-ml-dev-census-regression-datalab/training/model/
gs://cloud-ml-dev-census-regression-datalab/training/staging/
gs://cloud-ml-dev-census-regression-datalab/training/train/


Online Prediction using ML Engine
==========

We first have to deploy the model, and then we can make requests on it.

In [6]:
!gcloud beta ml models create census_model --regions=us-central1
!gcloud beta ml versions create v1 --model census_model --origin {CLOUD_ROOT}/training/model

Creating version (this might take a few minutes)......done.


In [7]:
sd.cloud_predict(
  model_name='census_model',
  model_version='v1',
  data=['490,64,2,0,1,0,2,8090,015,01,1,00590,00500,1,18,0,2,1',
        '1225,32,5,0,4,5301,2,9680,015,01,1,00100,00100,1,21,2,1,1',
        '1226,30,1,0,1,0,2,8680,020,01,1,00100,00100,1,16,0,2,1']
)

,key_from_input,predicted_target
0,490,20.6305
1,1225,59.4538
2,1226,20.8643


In [8]:
sd.cloud_predict(
  model_name='census_model',
  model_version='v1',
  data=pd.DataFrame(
    [[490,64,2,0,1,0,2,8090,"015","01",1,"00590","00500",1,18,0,2,1],
     [1225,32,5,0,4,5301,2,9680,"015","01",1,"00100","00100",1,21,2,1,1],
     [1226,30,1,0,1,0,2,8680,"020","01",1,"00100","00100",1,16,0,2,1]])
)

,key_from_input,predicted_target
0,490,20.6305
1,1225,59.4538
2,1226,20.8643


Cleaning things up
=====

If you want to delete the files you made on GCS, uncomment and run the next cell.

In [9]:
#!gsutil rm -fr {CLOUD_ROOT}

To delete the models created, uncomment and run the next cell.

In [10]:
#!gcloud beta ml versions delete v1 --model census_model 
#!gcloud beta ml models delete census_model 